In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms as T
from torchvision.transforms import v2

In [ ]:
train_transform = v2.Compose([
    v2.RandomResizedCrop([320,320], scale=[0.25,1.0], ratio=[1.0,1.0]),
    #v2.RandomRotation(180),
    v2.RandomPerspective(),
    v2.RandomHorizontalFlip(0.5),
    #RandomVerticalFlip(0.5)
    v2.ToTensor(),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = v2.Compose([
  v2.Resize([320,320]),
  v2.ToTensor(),
  v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

C:\Users\erhan\AppData\Roaming\Python\Python311\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Downloading Flowers102 dataset from datasets

In [ ]:
training_data = datasets.Flowers102(root="dataset", split="train", transform=train_transform, download=True)

test_data = datasets.Flowers102(root="dataset", split="val", transform=test_transform, download=True)

In [ ]:
batch_size = 64

#Create data loaders
train_dataloader = DataLoader(training_data, batch_size = batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 3, 320, 320])
Shape of y: torch.Size([64]) torch.int64


image mode: RGB
varying image size


In [ ]:
#Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

#Define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.model = nn.Sequential(
      nn.Conv2d(3, 16, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Conv2d(16, 32, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Conv2d(32, 64, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Conv2d(64, 128, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Flatten(),
      nn.Linear(128 * 40 * 40, 1024),
      nn.ReLU(),
      nn.Linear(1024, 512),
      nn.ReLU(),
      nn.Linear(512,256),
      nn.ReLU(),
      nn.Linear(256,102)
    )

  def forward(self, x):
    return self.model(x)

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (model): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): Flatten(start_dim=1, end_dim=-1)
    (12): Linear(in_features=204800, out_features=1024, bias=True)
    (13): ReLU()
    (14): Linear(in_features=1024, out_features=512, bias=True)
    (15): ReLU()
    (16): Linear(in_features=512, out_features=256, bias=True)
    (17): ReLU()
    (18): Linear(in_features=256, out_f

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.005)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    #Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 5 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct) :>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 60
for t in range(epochs):
  print(f"Epoch {t+1} \n ---------------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!!!!")

Epoch 1 
 ---------------------------------------
loss: 4.620936 [   64/ 1020]
loss: 4.622182 [  384/ 1020]
loss: 4.629573 [  704/ 1020]
loss: 4.633657 [  960/ 1020]
Test Error: 
 Accuracy: 1.0%, Avg loss: 4.625380 

Epoch 2 
 ---------------------------------------
loss: 4.618210 [   64/ 1020]
loss: 4.620784 [  384/ 1020]
loss: 4.633463 [  704/ 1020]
loss: 4.634877 [  960/ 1020]
Test Error: 
 Accuracy: 1.0%, Avg loss: 4.625150 

Epoch 3 
 ---------------------------------------
loss: 4.625336 [   64/ 1020]
loss: 4.632495 [  384/ 1020]
loss: 4.624060 [  704/ 1020]
loss: 4.625289 [  960/ 1020]
Test Error: 
 Accuracy: 1.1%, Avg loss: 4.585633 

Epoch 4 
 ---------------------------------------
loss: 4.603209 [   64/ 1020]
loss: 4.579970 [  384/ 1020]
loss: 4.468671 [  704/ 1020]
loss: 4.488828 [  960/ 1020]
Test Error: 
 Accuracy: 1.9%, Avg loss: 4.427902 

Epoch 5 
 ---------------------------------------
loss: 4.366881 [   64/ 1020]
loss: 4.444664 [  384/ 1020]
loss: 4.291485 [  704/ 1

KeyboardInterrupt: 